In [1]:
%pip install -U pandasql

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 4.1 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 539.9/539.9 KB 6.8 MB/s eta 0:00:007.1 MB/s eta 0:00:01
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26784 sha256=bc1a901306bcc204cd2144a6f0967a382ec955bfeb3d58ad174a48e977f521fa
  Stored in directory: /home/davide/.cache/pip/wheels/e9/bc/3a/8434bdcccf5779e72894a9b24fecbdcaf97940607eaf4bcdf9
Successfully built pandasql
Note: you may need to restart the kernel to use updated packages.


In [1]:
from pandasql import sqldf
import pandas as pd

In [2]:
def execute_query(query: str):
    """
    Executes the input query on one of the datasets 
    saved in the globals() dictionary as variables.
    :param query: string representing the query to execute
    """
    return sqldf(query, globals())

In [3]:
# Import the data about AirBnBs in Austin, TX
listings = pd.read_csv("listings.csv")
calendar = pd.read_csv("calendar.csv")

## 2. Which places are the dirtiest based on users' reviews?
- host name and price\
(I'm going to make it complex by using a join between two tables obtained from the listing one)

In [11]:
# I'm intentionally decomposing listing table into 2 smaller
# tables so that I can perform a join operation to retrieve 
# the answer to last point of question n. 2
query3 = '''
    select host_id, host_url, host_name, host_is_superhost
    from listings
'''
hosts = execute_query(query3)

query4 = '''
    select id, host_id, name, description, review_scores_cleanliness
    from listings
    where review_scores_cleanliness is not NULL
    order by review_scores_cleanliness
    limit 10
'''
top10_dirtyplaces = execute_query(query4)

top10_dirtyplaces

,id,host_id,name,description,review_scores_cleanliness
0,340045,1725752,Greenbelt Professional Loft,<b>The space</b><br />Professionally furnished...,1.0
1,5086356,5658789,Bouldin Home for SXSW,"My place is walking distance to downtown, perf...",1.0
2,10671623,53170299,"Farmhouse In The City, Your SXSW Paradise","This quaint home was built in the 1800's, and ...",1.0
3,17604822,119647236,Cozy Apartment - 10-15 minutes from downtown!,"The apartment is really cozy, and really clean...",1.0
4,18501023,18153980,Key to Austin Riverside Two bedroom,"Spacious Town Home 1,026 square feet, Two Sto...",1.0
5,23399267,67794598,Perfect for SXSW DT stay with 2 too 6 friends,Dim quiet and tucked away down town condo easy...,1.0
6,23597868,121629892,Spacious SXSW Apartment Prime Location,Minutes from downtown. Walking distance to gro...,1.0
7,23629845,11004536,SoCo 1 br apartment near downtown,1 bedroom apartment 3 blocks from Congress in ...,1.0
8,23663215,66614618,3 Bedroom Near Downtown in a Quiet Neighborhood,3 bedroom house on the east side of Austin les...,1.0
9,28988724,60791915,"clear your mind, and stay here","the decor is quite basic and modern, everythin...",1.0


In [12]:
query5 = '''
    select t1.host_id, t1.host_name, t2.id, t2.name
    from hosts as t1 join top10_dirtyplaces as t2 on t1.host_id = t2.host_id
'''
execute_query(query5)

,host_id,host_name,id,name
0,1725752,Shanti,340045,Greenbelt Professional Loft
1,5658789,Ingrid,5086356,Bouldin Home for SXSW
2,18153980,Kristine,18501023,Key to Austin Riverside Two bedroom
3,53170299,Catherine,10671623,"Farmhouse In The City, Your SXSW Paradise"
4,119647236,Jonathan,17604822,Cozy Apartment - 10-15 minutes from downtown!
5,18153980,Kristine,18501023,Key to Austin Riverside Two bedroom
6,18153980,Kristine,18501023,Key to Austin Riverside Two bedroom
7,67794598,Tim,23399267,Perfect for SXSW DT stay with 2 too 6 friends
8,121629892,Robert,23597868,Spacious SXSW Apartment Prime Location
9,11004536,Victoria,23629845,SoCo 1 br apartment near downtown


## 1. How much are the top earners making?
- the next 30, 60, 90 days
- the next year

In [4]:
# View to make answering of question 1 easier.
query1 = '''
    select host_id, host_name, host_is_superhost,
           CAST(REPLACE(price, "$", "") AS INT) as clean_price,
           30 - availability_30 as booked_30,
           60 - availability_60 as booked_60,
           90 - availability_90 as booked_90,
           365 - availability_365 as booked_year
    from listings
'''
view_question1 = execute_query(query1)
view_question1

,host_id,host_name,host_is_superhost,clean_price,booked_30,booked_60,booked_90,booked_year
0,8028,Sylvia,t,111,24,30,37,56
1,8186,Elizabeth,f,39,23,32,32,307
2,13879,Todd,t,66,30,60,90,365
3,14156,Amy,t,149,27,46,76,275
4,1205884,Luke And Rachel,f,750,30,60,90,365
...,...,...,...,...,...,...,...,...
17066,439978004,Taylor,f,163,13,25,28,42
17067,256520767,Pearl,f,270,17,27,32,70
17068,84104364,Angel And Barbara,f,56,9,30,60,335
17069,79504097,Charlene,t,254,16,39,50,325


In [13]:
# Answer to question 1
query2 = '''
    select host_id, host_name,
           clean_price * booked_30 as earnings_30,
           clean_price * booked_60 as earnings_60,
           clean_price * booked_90 as earnings_90,
           clean_price * booked_year as earnings_year
    from view_question1
    order by earnings_30 desc
    limit 10
'''
execute_query(query2)

,host_id,host_name,earnings_30,earnings_60,earnings_90,earnings_year
0,20083288,Ben,29970,59940,89910,364635
1,19083042,Steven,29970,59940,89910,364635
2,27283435,Simeon,29970,59940,89910,364635
3,22899175,Jacob,29970,59940,89910,364635
4,58104709,Brian,29970,59940,89910,364635
5,111812829,Raymond,29970,59940,89910,231768
6,146541257,Rob,29970,59940,89910,330669
7,28477574,Scott,29970,59940,89910,364635
8,132086717,Corinne,29850,59700,89550,202980
9,175515738,Amelia,29850,59700,89550,363175
